In [1]:
import numpy as np
import sympy as sp
from sympy.solvers import solve

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn-colorblind')
plt.style.use('seaborn-talk')

/tmp/ipykernel_1788/3105660042.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')
/tmp/ipykernel_1788/3105660042.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


# Minimizing a quadratic objective under constraint $\left|x\right|+\left|y\right|\leq 1$

This notebook implements the minimization of the cost function:
$$f(\vec v) = \frac{1}{2} \vec v^t H \vec v + \vec b^t \vec v$$
subject to $$\left|x\right|+\left|x\right|\leq 1$$
where $\vec v = (x, y)$

The first part describes how the process is done and defines necessary functions and variables. 

In the second part (section [Interactive widget](#iwidget)), an interactive plot allows you to change values in $H$, $b$, and the multipliers $\mu_i$ and get a visualization of $f$, $L$, the found solution(s) and the gradient at the point(s) respecting the KKT conditions.

# Part I: Finding the KKT points with `sympy`
Hereafter we'll use a symbolic calculation engine (`sympy`) to compute all the analytical steps needed to solve such a constrained problem, based on th KKT optimality conditions:

1. Write the Lagrangean (using symbolic variables)
1. Compute $\vec\nabla L$
1. Find the set of solutions to the system defined by KKT stationarity and complementary slackness:
$$\vec\nabla L = 0$$ 
$$\vec\mu_i g_i = 0\text{, for }i\in \{1,2,3,4\}$$
4. Select the solutions that respect the primal and dual feasibility conditions:
$$g_i\leq 0\text{, for }i\in \{1,2,3,4\}$$
$$\mu_i\geq 0\text{, for }i\in \{1,2,3,4\}$$

## Symbolic declarations

### function $f$

In [2]:
from sympy.matrices import Matrix

h11, h12, h22 = sp.symbols('h11 h12 h22', real=True)
H = Matrix([[h11, h12],[h12, h22]])

b1, b2 = sp.symbols('b1 b2', real=True)
b = Matrix([b1, b2])

x, y = sp.symbols('x y', real=True)
xy = Matrix([x, y])
f = ((1/2)*xy.T @ H @ xy + b.T @ xy)[0]
grad_f = Matrix([sp.diff(f, var) for var in [x,y]])

### constraints $g_1$ through $g_4$

In [3]:
g1=x+y-1
g2=x-y-1
g3=-x-y-1
g4=-x+y-1

g=Matrix([g1,g2,g3,g4])

grad_g1 = Matrix([sp.diff(g1, var) for var in [x,y]])
grad_g2 = Matrix([sp.diff(g2, var) for var in [x,y]])
grad_g3 = Matrix([sp.diff(g3, var) for var in [x,y]])
grad_g4 = Matrix([sp.diff(g4, var) for var in [x,y]])


### Lagrangean

In [4]:
# declare multipliers as variables u1 to u4
u1,u2, u3, u4 = sp.symbols('u1 u2 u3 u4', real=True)
u = Matrix([u1, u2, u3, u4])
# write the L = f + u dot g
L = f + u.dot(g)
# compute L gradient
grad_L = Matrix([sp.diff(L, var) for var in [x,y]])

In [5]:
display (L)

b1*x + b2*y + u1*(x + y - 1) + u2*(x - y - 1) + u3*(-x - y - 1) + u4*(-x + y - 1) + x*(0.5*h11*x + 0.5*h12*y) + y*(0.5*h12*x + 0.5*h22*y)

In [6]:
display(grad_L)

Matrix([
[b1 + 1.0*h11*x + 1.0*h12*y + u1 + u2 - u3 - u4],
[b2 + 1.0*h12*x + 1.0*h22*y + u1 - u2 - u3 + u4]])

## Solve for KKT's stationarity and complementary slackness

The system to be solved here may be underdetermed and complex to solve while keeping the coeficients in $H$ and $\vec b$ completely free.
 We will first choose values for these coeficients before we try to find solutions to the system



**TODO** Complete the code so that $\vec b = [-3, 1/4]$ and $H = 2I$ (where $I$ is the identity matrix).

In [30]:
# Pick values for coeficients in H and b
# TODO complete the code bellow
subs = {b1: ..., b2:..., h11:..., h22:..., h12:... }

# Both get printed bellow
display(H.n(subs=subs))
display(b.n(subs=subs))

Matrix([
[2.0,   0],
[  0, 2.0]])

Matrix([
[ -3.0],
[-0.25]])

Now we use `sympy`'s function `nonlinsolve` to find solutions to the system

In [ ]:
# gather all equations in a list
system = list(grad_L.n(subs=subs)) + list(u.multiply_elementwise(g))
# give the list to sympy solver
sys_solutions = sp.nonlinsolve(system, (x, y, u1, u2, u3, u4))
display(sys_solutions)

{(-1.0, 0, 0, 0, -2.625, -2.375), (0, -1.0, 0, 0.375, -2.625, 0), (0, 1.0, 0.625, 0, 0, -2.375), (0.1875, -1.1875, 0, 0, -2.625, 0), (0.3125, 1.3125, 0, 0, 0, -2.375), (1.0, 0, 0.625, 0.375, 0, 0), (1.1875, -0.1875, 0.625, 0, 0, 0), (1.3125, 0.3125, 0, 0.375, 0, 0), (1.5, 0.125, 0, 0, 0, 0)}

To filter out the solutions, we need to check for KKT`s feasibility conditions:
1. Solution $(x,y)$ must be in the feasible set, i.e., all $g_i\leq 0$
2. Multipliers must be all non negative, i.e., all $\mu_i\geq 0$

We'll do so with the following function:

In [9]:
def solve_kkt(subs):
    # solve the (non-)linear system of equations
    system = list(grad_L.n(subs=subs)) + list(u.multiply_elementwise(g))
    sys_solutions = sp.nonlinsolve(system, (x,y, u1, u2, u3, u4))
    # filter for dual feasibility (all  u >= 0)
    eps = 1e-15
    dual_feasible = list(filter(lambda s: all(map(lambda u: u>=-eps,  s[2:])), sys_solutions))
    # filter for primal feasibility (all  g_i >= 0)
    def is_primal_feasible(sol):
        return all(map(
            lambda gi: gi.evalf(subs={x:sol[0], y:sol[1]}) <= eps, g))

    primal_feasible = list(filter(is_primal_feasible, dual_feasible))
    return primal_feasible

If we call the function (using the same $H$ and $\vec b$ defined by `subs`) we get that a single point respects all conditions.

In [10]:
solve_kkt(subs)

[(1.0, 0, 0.625, 0.375, 0, 0)]

**QUESTION**: Is this point a global minimum? Justify your answer.

# <a name='iwidget'></a> Part II: Interactive widget 
Run the cells bellow to have the interactive widget displayed to you at the end of the notebook.

In [46]:
# select a x and y range for the contour plat, 
# as well as a delta for the step between points
xlim = -5,5
ylim = -5,5
# delta=0.25
# xspace = np.arange(*xlim,delta)
# yspace = np.arange(*ylim,delta)
# ow instead of delta, select a number of points
npts=100
xspace = np.linspace(*xlim,npts)
yspace = np.linspace(*ylim,npts)

# Create a mesh for contour plot
X, Y = np.meshgrid(xspace, yspace)

In [47]:
def np_f(x, y, subs):
    # rewriting the expression for f so the numerical calculations 
    # needed for contour plots are faster 
    (h11, h12), ( _, h22) = np.array(H.n(subs=subs), dtype=float)
    (b1, b2)= np.array(b.n(subs=subs), dtype=float)
    return  b1*x + b2*y + x*(0.5*h11*x + 0.5*h12*y) + y*(0.5*h12*x + 0.5*h22*y)

def np_g(x, y):
    # rewriting the expressions for g_i so the numerical calculations 
    # needed for contour plots are faster 
    g1=x+y-1
    g2=x-y-1
    g3=-x-y-1
    g4=-x+y-1

    g=np.stack([g1,g2,g3,g4], axis=0)
    return g

In [51]:
from matplotlib.patches import Polygon

def replot(mus, subs, ax, cax=None):
    plt.sca(ax)
    fig = plt.gcf()

    F = np_f(X, Y, subs)
    GS = np_g(X, Y)
    Z = F + mus @ GS.swapaxes(0,1)

    c = ax.contourf(X, Y, F , cmap='magma', alpha=0.7, )
    cbar = fig.colorbar(c, cax=cax, ax=ax) 
    cax = cax or cbar.ax
    cax.set_ylabel('f(x,y) values')
    cax.margins(tight=True)
    # ax.clabel(c, colors='w')

    ax.set_title('Lagrangean - contours')
    c = ax.contour(X, Y, Z, cmap='viridis_r')
    ax.clabel(c)

    # draw the feasible region
    corners = [[1,0],[0,-1],[-1,0],[0,1],[1,0]]
    ax.add_patch(Polygon(
        corners, alpha=0.3, label='Feasible set',
        facecolor='cyan', edgecolor='cyan', ls=":", lw=3))
    # ax.add_patch(Circle((0,0), 1))
    
    for i in range(4):
        p1 = corners[i]
        p2 = corners[i+1]
        plt.plot(*zip(p1,p2),':', color='blue', alpha=0.5)

    # plot solutions and gradient at solution
    solution = solve_kkt(subs)

    for sol in solution:
        x_, y_ = [float(i) for i in sol[0:2]]
        plt.plot(x_,y_, '*', color='w')
        mu = [float(s) for s in sol[2:]]
        plt.annotate(
            f'$\\vec\mu$={np.round(mu,3)}',xy=(x_,y_,), 
            xytext=(4, 4), textcoords='offset points',
            fontsize=12, color='white')
        grads_gi = [grad_g1, grad_g2, grad_g3, grad_g4]
        for i, (mu_i, grad_gi) in enumerate(zip(mu, grads_gi)):
            # draw gradient for active constraint
            if mu_i != 0:
                gx, gy = np.array(grad_gi.n(subs=subs), dtype=float) 
                ax.annotate(f'$\\mu_{i+1}\\nabla g_{i+1}$', xy=(x_,y_,), 
                            xytext=(mu_i*gx+x_, mu_i*gy+y_), fontsize=12,
                            arrowprops=dict(arrowstyle="<|-", color='w'),
                            color='w')
            # draw gradient for f
            gfx, gfy = np.array(grad_f.n(subs=subs).n(subs={x:x_, y:y_}), dtype=float)

            ax.annotate(
                f'$\\nabla f$', xy=(x_,y_,), xytext=(gfx+x_, gfy+y_),
                fontsize=12, color='w',
                arrowprops=dict(arrowstyle="<|-", color='w'))
    if solution:
        plt.plot(x_,y_, '*', color='w', label="L's stationary points")

    # final adjustments in axis
    plt.axis('equal')
    ax.spines[['left', 'bottom']].set_position('zero')
    ax.spines[['top', 'right']].set_visible(False)
    ax.tick_params(axis='both', which='major', labelsize=9)

    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.margins(tight=True)
    plt.legend(framealpha=0.2)


# %matplotlib inline
# mus = np.zeros(4)

# fig, ax = plt.subplots(figsize=(14,8))
# replot(mus, subs, ax=ax)

In [ ]:
%matplotlib widget
from matplotlib.widgets import Slider, Button, RadioButtons, TextBox

fig, ax = plt.subplots(figsize=(13,10))
plt.subplots_adjust(bottom=0.2, right=0.75)
# ax for colorbar
cax = plt.axes([0.8, 0.2, 0.03, 0.7])
# ax for sliders
axcolor = 'lightgoldenrodyellow'

# setting up sliders
mu_sliders = []
for i in range(u.shape[0]):
    axmu = plt.axes([0.3, 0.13-i*0.04, 0.55, 0.03], facecolor=axcolor)
    mu0 = 0
    mu_values = np.arange(-3,3,0.125)
    smu = Slider(axmu, '$\mu$'+f'{i+1}', mu_values.min(), mu_values.max(), valinit=mu0, valstep=mu_values)
    mu_sliders.append(smu)

# set up boxes for h and b values
# h first column
left = 0.05
axh11 = plt.axes([left, 0.1, 0.04, 0.05])
bh11 = TextBox(axh11,label='$h_{11}$', initial=2, textalignment='center')
axh12 = plt.axes([left, 0.05, 0.04, 0.05])
bh12 = TextBox(axh12,label='$h_{12}$', initial=0, textalignment='center')
# h second column
left +=0.07
axh22 = plt.axes([left, 0.05, 0.04, 0.05])
bh22 = TextBox(axh22,label='$h_{22}$', initial=2, textalignment='center')
# ax to repeat h12 as h21
axh21 = plt.axes([left, 0.1, 0.04, 0.05])
# 3rd columns: boxes for b
left += 0.07
axb1 = plt.axes([left, 0.1, 0.04, 0.05])
bb1 = TextBox(axb1,label='$b_{1}$', initial=-3, textalignment='center')
axb2 = plt.axes([left, 0.05, 0.04, 0.05])
bb2 = TextBox(axb2,label='$b_{2}$', initial=1/4, textalignment='center')


# monitor slider and update
def update(val):
    axh21.clear()
    axh21.axis('off')
    axh21.text(0.01,0.5,"$h_{21}=$"+f"{bh12.text}")

    mus = np.array([smu.val for smu in mu_sliders])
    subs = {b1:eval(bb1.text), b2:eval(bb2.text), h11:eval(bh11.text), 
            h22:eval(bh22.text), h12:eval(bh12.text)}
    # redraw plot
    ax.clear()
    cax.clear()
    replot(mus,subs, ax, cax)
# do the first plot
update(None)

for element in mu_sliders:
    element.on_changed(update)

for textbox in [bh11, bh12, bh22, bb1, bb2]:
    textbox.on_submit(update)


plt.show()

## Exercises wtih the interactive widget
### Exercise 1
Set values of b such that:
1. only the constraint  $g_1$ is active.
2. only constraints  $g_1$ and  $g_2$ are active.
3. only the constraint  $ g_2$ is active.
4. only constraints  $ g_2$ and  $g_3$ are active.
5. only the constraint  $ g_3$ is active.
6. only constraints  $g_3$ and  $g_4$ are active.
7. only the constraint  $ g_4$ is active.
8. only constraints  $g_4$ and  $g_1$ are active.
9. no constraint is active.

For each case:
- use the sliders for $\mu_i$ to match the multiplier values to those of the found solution.
- observe the contours of the Lagrangean moving towards the active constraints.
- note which multipliers are non-null in each case.

### Exercise 2
1. Set values of H such that:
    - H is negative definite
    - H is indefinite
2. Change b values to move the quadratic form around the feasible region.
3. Note how the KKT conditions will sometimes lead to multiple solutions, sometimes to a single solution, and sometimes to no solution at all.
    - Can you interpret what is happening in each case?